<a href="https://colab.research.google.com/github/pablohenrique93/projeto-treino-etl-credit-card-transactions-india/blob/main/etl_credit_card_transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação das Bibliotecas

In [ ]:
pip install pandera

In [ ]:
pip install pymongo

#Importação das Bibliotecas

In [ ]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient

In [ ]:
# Importação de Bibliotecas para Visualização de Dados
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Carregamento da Base de Dados

In [ ]:
df = pd.read_csv('/content/Credit card transactions - India - Simple.csv')

In [ ]:
# Exibindo o dataframe
df

# Backup do Dataframe

In [ ]:
# Faz-se necessário realizar um backup do dataframe carregado antes de qualquer manipulação de dados, em caso de algum erro
# nos processos posteriores, os dados estarão assegurados
backup_df = df.copy

# Pré Análise do Dataframe

In [ ]:
# Exibindo as 10 primeiras linhas do dataframe
df.head(10)

In [ ]:
# Exibindo as 10 últimas linhas do dataframe
df.tail(10)

In [ ]:
#Verificando os tipos das colunas do dataframe
df.dtypes

In [ ]:
# Verificando o tamanho do Dataframe
df.shape

# Tratamento da Base de Dados

In [ ]:
#Verificação das colunas com o objetivo de encontrar inconsistências
pd.unique(df['City'])

In [ ]:
pd.unique(df['Date'])

In [ ]:
pd.unique(df['Card Type'])

In [ ]:
pd.unique(df['Exp Type'])

In [ ]:
pd.unique(df['Gender'])

In [ ]:
pd.unique(df['Amount'])

# Limpeza de Inconsistências e Padronização de Colunas

In [ ]:
df.dtypes

In [ ]:
# Renomeação e tradução de colunas, utilizando a fonte dos dados como parâmetro de tradução
# Procedimento realizado com o objetivo de facilitar a compreensão dos registros

df.rename(columns={
    'City':'cidade_india',
    'Date':'data',
    'Card Type':'tipo_cartao',
    'Exp Type':'tipo_despesa',
    'Gender':'genero_titular',
    'Amount':'valor'
},inplace=True)

In [ ]:
# Conferindo dataframe após manipulação
df

In [ ]:
# Como exibido anteriormente os dados deste dataset são exclusivamente da Índia
# A coluna 'cidade_india', por sua vez, possui o termo 'India' ao lado de cada registro de cidade, se fazendo desnecessário.
# Neste caso droparemos este termo dos registros, para que a coluna permaneça somente com o nome da cidade, padronizando a mesma 

df['cidade_india'] = df['cidade_india'].str.replace(', India', '')

In [ ]:
# Convertendo o tipo da coluna Data, de String para Date
df['data'] = pd.to_datetime(df['data'], format='%d-%b-%y')

In [ ]:
# Exibindo o dataframe após alteração
df

In [ ]:
# Renomeando e traduzindo registros da coluna 'tipo_despesa' 
df.replace({'tipo_despesa':'Bills'}, 'contas',regex=True, inplace=True)  
df.replace({'tipo_despesa':'Food'}, 'alimentacao',regex=True, inplace=True)  
df.replace({'tipo_despesa':'Entertainment'}, 'entretenimento',regex=True, inplace=True)
df.replace({'tipo_despesa':'Grocery'}, 'mercado',regex=True, inplace=True)    
df.replace({'tipo_despesa':'Fuel'}, 'combustivel',regex=True, inplace=True)
df.replace({'tipo_despesa':'Travel'}, 'viagem',regex=True, inplace=True)

In [ ]:
#Exibindo o dataframe após alteração
df

In [ ]:
# Verificando se o dataframe possui dados faltantes, ou seja, nulls
df.isnull().sum()
# Como verificado, o dataframe não possui dados faltantes

In [ ]:
# Verificando a existência de dados duplicados
duplicados = df.duplicated()
num_duplicados = duplicados.sum()
print(num_duplicados)
# Como podemos observar, não existe linhas duplicadas neste dataframe

# Visualização de Dados, Insights, Plots, Groupbys

In [ ]:
# Groupby para verificação da quantidade de gêneros de cada titular do cartão
df.groupby(['genero_titular']).size().sort_values(ascending=False)
# Como podemos observar, temos mais mulheres do que homens

In [ ]:
# Plot seaborn para visualização dos registros anteriores, o ";" é utilizado no final do código para exibir um gráfico "mais limpo"
sns.countplot(x=df['genero_titular']);

In [ ]:
# Groupby com auxílio de Plot para verificação dos maiores gastos registrados neste dataframe
df.groupby(['tipo_despesa']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='tipo_despesa', ylabel = 'quantidade', rot=90);
# Como podemos observar, os maiores gastos foram com alimentação, seguido de combustível e contas

In [ ]:
df.groupby(['data']).size().sort_values(ascending=False)

In [ ]:
# Plotagem dos tipos de despesa por gênero do titular do cartão
grafico = px.treemap(df, path=['tipo_despesa', 'genero_titular']);
grafico.show()
# Como podemos observar, a mulheres lideram em gastos por tipo de despesa, ficando atrás somente no gasto em combustível

In [ ]:
# Plotagem por cidade, tipo do cartão, gênero do titular e valor gasto
grafico = px.treemap(df, path=['cidade_india', 'tipo_cartao', 'genero_titular', 'valor']);
grafico.show() 
# É possível notar que as maiores incidências de gastos ocorrem nas cidades de Bengaluru, Greater Mumbai e Ahmedabad

In [ ]:
# Plot dos cartões mais utilizados neste período
sns.countplot(x=df['tipo_cartao']);
# Podemos observar que os três mais utilizados foram o Silver, Signature e Platinum

In [ ]:
# Cálculo do valor total gasto por 'genero_titular' usando o método groupby e sum:
df_genero_valor = df.groupby('genero_titular')['valor'].sum().reset_index()
# Plot do valor total gasto por gênero
sns.barplot(x='genero_titular', y='valor', data=df_genero_valor);
# É possível confirmar através desta soma, que as mulheres consumiram mais do que os homens neste período

In [ ]:
# Plot para verificação das datas em que mais ocorreram maiores índices de gasto
fig = px.bar(df,x="data", y="valor", title='Índices de Datas e seus Respectivos Volumes de Gastos')
fig.show()
# Podemos notar que as datas em que mais ocorreram gastos foram em 07/10/2014, 20/09/2014 e 11/08/2014

# Preparação dos Arquivos

In [ ]:
# Salvando dataset tratado localmente, em formato 'csv'

df.to_csv('credit_card_transactions_india_treated.csv', index=False)

#Envio para o MongoDB

In [ ]:
# Criando conexão com o MongoDB

uri = 'mongo_uri'
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='mongokey')

In [ ]:
# Criando coleção para enviar para o MongoDB

db = client['credit_india']
colecaotreated = db['credit_card_india_transactions_treated']

In [ ]:
# Enviando o Dataset tratado para o MongoDB

df.reset_index(drop=True)
df01 = df.to_dict("records")
colecaotreated.insert_many(df01)

In [ ]:
# Verificando se o arquivo foi enviado com sucesso
colecaotreated.count_documents({})
# Como podemos conferir o dataset foi enviado corretamente para o MongoDB

#FIM DO PROCESSO